In [69]:
import pandas as pd
import pyodbc 

In [70]:
df_cases = pd.read_parquet('./Processed/WHO-COVID19-CASES.parquet')

In [71]:
max_df_date = df_cases['DateReported'].max().date()

In [27]:
# df_list = df_cases.values.tolist()

In [72]:
server = 'lirkovsrv.database.windows.net'
database = 'lirkovdb'
username = 'lirkov'
password = '{LLirkoff31}'   
driver= '{ODBC Driver 17 for SQL Server}'

In [82]:
conn =  pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

cursor.execute('SELECT MAX(DateReported) FROM covid.who_cases')
max_db_date = cursor.fetchone()

if max_db_date[0] < max_df_date:
    df_upsert = df_cases[df_cases['DateReported'].dt.date > max_db_date[0]]
    for index, row in df_upsert.iterrows():
        cursor.execute("INSERT INTO covid.who_cases (DateReported, Country, IsoCode, WhoRegion, NewCases, CumulativeCases, NewDeaths, CumulativeDeaths) VALUES(?,?,?,?,?,?,?,?)", row.DateReported, row.Country, row.IsoCode, row.WhoRegion, row.NewCases, row.CumulativeCases, row.NewDeaths, row.CumulativeDeaths)
else:
    print('Database is up to date')


conn.commit()
cursor.close()

In [65]:
if max_db_date[0] < max_df_date:
    

True